In [3]:
import pickle
import sys
import os
import json
import pandas as pd
import ast
import numpy as np
from utils.beh_functions import parseSessionID

In [4]:
# Load the CCF summary CSV file
ccf_file_path = '/root/capsule/code/data_management/combined_ephys_session_ccf_summary.csv'
ccf_df = pd.read_csv(ccf_file_path)
# load the combined pickle file
combined_pickle_path = '/root/capsule/scratch/combined/combine_unit_tbl/combined_unit_tbl.pkl'
with open(combined_pickle_path, 'rb') as f:
    combined_unit_tbl = pickle.load(f)

In [5]:
skipping_ani_ids = ['751004', '751769']

In [11]:
for ind, row in ccf_df.iterrows():
    session = row['session']
    aniID, date_obj, raw_id = parseSessionID(session)
    if aniID not in skipping_ani_ids:
        units = ast.literal_eval(row['opto_units']) if isinstance(row['opto_units'], str) else None
        ccfs = ast.literal_eval(row['opto_unit_ccf_pts']) if isinstance(row['opto_unit_ccf_pts'], str) else None
        if ccfs is not None:
            for unit, ccf in zip(units, ccfs):
                if len(combined_unit_tbl[(combined_unit_tbl['unit'] == unit) & (combined_unit_tbl['session'] == session)]) > 0:
                    combined_unit_tbl.loc[(combined_unit_tbl['unit'] == unit) & (combined_unit_tbl['session'] == session), 'x_ccf'] = ccf[0]
                    combined_unit_tbl.loc[(combined_unit_tbl['unit'] == unit) & (combined_unit_tbl['session'] == session), 'y_ccf'] = ccf[1]
                    combined_unit_tbl.loc[(combined_unit_tbl['unit'] == unit) & (combined_unit_tbl['session'] == session), 'z_ccf'] = ccf[2]
                else:
                    print(f"Unit {unit} in session {session} not found in combined_unit_tbl. Skipping update.")

In [ ]:
# update the existing pickle file with the new CCF coordinates
with open(combined_pickle_path, 'wb') as f:
    pickle.dump(combined_unit_tbl, f)